In [1]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats
import matplotlib.pyplot as plt


In [2]:
path = "ExperimentData"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/ResponsiveAR/Experiment/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/ResponsiveAR/Experiment/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

user_group_mapping = {}

for i in range(30, 87):
    user_group_mapping[i] = "A"
    if(i % 2 != 0):
        user_group_mapping[i] = "B"


    #special cases
    if(i == 79):
        user_group_mapping[i] = "A"

print(user_group_mapping)

Files and directories in ' ExperimentData ' :
['61', '59', '68', '57', '32', '35', '69', '56', '51', '58', '67', '60', '34', '33', '.DS_Store', '45', '80', '86', '44', '43', '38', '36', '31', '65', '54', '53', '30', '37', '39', '55', '63', '64', '46', '79', '41', '83', '48', '70', '84', '85', '71', '82', '49', '40', '47', '78']
['ExperimentData/61/ResponsiveAR/Experiment/', 'ExperimentData/59/ResponsiveAR/Experiment/', 'ExperimentData/68/ResponsiveAR/Experiment/', 'ExperimentData/57/ResponsiveAR/Experiment/', 'ExperimentData/32/ResponsiveAR/Experiment/', 'ExperimentData/35/ResponsiveAR/Experiment/', 'ExperimentData/69/ResponsiveAR/Experiment/', 'ExperimentData/56/ResponsiveAR/Experiment/', 'ExperimentData/51/ResponsiveAR/Experiment/', 'ExperimentData/58/ResponsiveAR/Experiment/', 'ExperimentData/67/ResponsiveAR/Experiment/', 'ExperimentData/60/ResponsiveAR/Experiment/', 'ExperimentData/34/ResponsiveAR/Experiment/', 'ExperimentData/33/ResponsiveAR/Experiment/', 'ExperimentData/45/Respon

In [3]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [4]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [5]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1653325550021.json
ExperimentSession_1_1653325550021.json
ExperimentSession_2_1653325550021.json
ExperimentSession_3_1653325550021.json
ExperimentSession_4_1653325550021.json
ExperimentSession_5_1653325550021.json
ExperimentSession_6_1653325550021.json
ExperimentSession_7_1653325550021.json
ExperimentSession_8_1653325550021.json
ExperimentSession_9_1653325550021.json
ExperimentSession_0_1652910130460.json
ExperimentSession_1_1652910130460.json
ExperimentSession_2_1652910130460.json
ExperimentSession_3_1652910130460.json
ExperimentSession_4_1652910130460.json
ExperimentSession_5_1652910130460.json
ExperimentSession_6_1652910130460.json
ExperimentSession_7_1652910130460.json
ExperimentSession_8_1652910130460.json
ExperimentSession_9_1652910130460.json
ExperimentSession_0_1653072453927.json
ExperimentSession_1_1653072453927.json
ExperimentSession_2_1653072453927.json
ExperimentSession_3_1653072453927.json
ExperimentSession_4_1653072453927.json
ExperimentSession_5_16530

In [6]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."


USER 56 was unable to finish task 7 (when starting from 0) remove their task 8 from data
USER 85 was unable to finish task 7 and 8 (when starting from 0) remove their task 8 and 9 from data

In [7]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf.head(2)

(144426, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,61,1,1653325566491,16470,"{'x': -0.018050827085971832, 'y': -0.004202843...","{'x': -0.0035658455453813076, 'y': -0.24633377...","{'x': 0.12413376569747925, 'y': -0.00124909367...","{'x': 14.260669708251953, 'y': 359.78918457031...","{'x': -0.017374971881508827, 'y': -0.004261824...","{'x': -0.0014919827226549387, 'y': -0.24794106...","{'x': -0.0683116614818573, 'y': 0.569282412528...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653325566402, 'systemTime': '10...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."
1,61,2,1653325566591,16570,"{'x': -0.01778065226972103, 'y': -0.0040911692...","{'x': -0.007211289368569851, 'y': -0.244305133...","{'x': 0.1230895146727562, 'y': -0.003629593877...","{'x': 14.140772819519043, 'y': 359.57391357421...","{'x': -0.01789078675210476, 'y': -0.0043039829...","{'x': -0.006627660244703293, 'y': -0.244332477...","{'x': -0.059191010892391205, 'y': 0.6012426018...","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],"{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [10]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
users.head(2)

,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,61,1,1653325566491,16470,"{'x': -0.018050827085971832, 'y': -0.004202843...","{'x': -0.0035658455453813076, 'y': -0.24633377...","{'x': 0.12413376569747925, 'y': -0.00124909367...","{'x': 14.260669708251953, 'y': 359.78918457031...","{'x': -0.017374971881508827, 'y': -0.004261824...","{'x': -0.0014919827226549387, 'y': -0.24794106...","{'x': -0.0683116614818573, 'y': 0.569282412528...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653325566402, 'systemTime': '10...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."
65,61,66,1653325573122,23101,"{'x': -0.015795331448316574, 'y': 0.0082537140...","{'x': 0.013220150955021381, 'y': -0.1552816033...","{'x': 0.07786977291107178, 'y': 0.006738382391...","{'x': 8.933128356933594, 'y': 0.76678234338760...","{'x': -0.01348222978413105, 'y': 0.00690337689...","{'x': 0.009646527469158173, 'y': -0.1605786234...","{'x': 0.07619231939315796, 'y': -0.07652626186...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653325572982, 'systemTime': '10...","{'name': 'Article Display', 'scale': {'x': 0.2..."


In [11]:
experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID
        unixTime = users["unixTime"][i]

experimentDf.head(2)

(3560, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653325566402,10-06-06-40,True,instruction,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0
1,1653325572982,10-06-12-98,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0


In [12]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
1,1653325572982,10-06-12-98,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0
3,1652747526727,17-32-06-72,False,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.06302891671657562, 'y': 0.021540082991...",2nd Title,,34.0
5,1653347136313,16-05-36-31,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.021247876808047295, 'y': 0.05604220554...",2nd Title,,53.0


In [532]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."
120,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165..."
240,53,153,The task is to click on the 2nd article title....,1653347136263,1653347142364,6101,0,True,"[{'userID': 53, 'frameNum': 1, 'unixTime': 165..."
330,41,80,The task is to click on the 2nd article title....,1652904601496,1652904604747,3251,0,True,"[{'userID': 41, 'frameNum': 1, 'unixTime': 165..."
388,71,110,The task is to click on the 2nd article title....,1653342438648,1653342442018,3370,0,True,"[{'userID': 71, 'frameNum': 1, 'unixTime': 165..."
...,...,...,...,...,...,...,...,...,...
369,70,1,,0,0,0,9,False,"[{'userID': 70, 'frameNum': 2210, 'unixTime': ..."
119,60,1,,0,0,0,9,False,"[{'userID': 60, 'frameNum': 3456, 'unixTime': ..."
239,54,1,,0,0,0,9,False,"[{'userID': 54, 'frameNum': 2705, 'unixTime': ..."
39,57,1,,0,0,0,9,False,"[{'userID': 57, 'frameNum': 3809, 'unixTime': ..."


In [13]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982


In [14]:
endEvents = experimentDf.loc[(experimentDf["eventName"] == "answer") & (experimentDf["guess"] == experimentDf["correct_answer"])]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
93,1652899453664,11-44-13-66,False,answer,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.21994057297706604, 'y': -0.01243835873...",2nd Title,2nd Title,56.0


In [15]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792


In [16]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([ 9,  4,  0,  0, 10,  0, 10,  0, 10, 10, 16, 10, 17,  0, 10,  9, 10,
        0, 10,  0,  6,  0,  0, 10,  4,  0,  0, 10, 10,  0,  2,  0, 10,  0,
       10,  0,  0,  0, 10,  7, 10, 10, 12, 11,  0, 15, 10, 10, 11,  0,  0,
       10,  0,  0,  0, 10,  0,  9, 10,  9,  0,  0,  0, 10,  0, 10,  0,  0,
        0,  4, 10,  0, 11,  0,  0,  0,  0,  0,  5, 11,  6,  9, 10,  0, 10,
        9, 10,  0, 10,  0,  0,  0,  0,  0,  0, 10, 10,  0, 10,  0,  9, 10,
       10,  0, 10,  0, 11, 10,  0, 10,  0,  0,  0, 10, 10, 12,  0,  4,  0,
        1, 10, 11, 10,  0, 10, 10, 10, 14,  0, 11,  5, 11,  9,  9,  0, 10,
        9,  9,  3,  0,  0,  9, 10, 10,  0, 10, 10,  0, 10, 10, 10,  0, 10,
       10,  0,  5,  0,  0,  0, 10,  3, 10,  0,  0,  0,  9, 10,  0, 11, 10,
        0,  0,  0,  0, 11, 10, 11,  0, 10,  0,  0,  0, 11, 10, 10,  0,  0,
        0,  0,  0,  2,  0,  0, 10, 12,  4,  1,  0, 10,  0, 10, 10,  0,  0,
        0, 10,  0, 10,  0,  0,  0, 10, 10, 11,  0, 10,  0,  0,  0,  0, 10,
        0,  0,  0,  0,  0

In [17]:
guessEvents = experimentDf.loc[experimentDf["eventName"] == "answer"]
guessEvents.head(3)

guessEvents = guessEvents.sort_values("unixTime")
guessEvents.head(9)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
161,1652721833975,10-23-53-97,False,answer,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.05843022093176842, 'y': -0.00664969347...",2nd Title,2nd Title,30.0
393,1652721852805,10-24-12-80,False,answer,1,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.0025609361473470926, 'y': 0.027374526...",5th Title,5th Title,30.0
564,1652721877306,10-24-37-30,False,answer,2,Article Display,"{'x': 0.3499999940395355, 'y': 0.3499999940395...","{'x': -0.03865040838718414, 'y': -0.0744887813...",4th Author,4th Author,30.0
809,1652721898906,10-24-58-90,False,answer,3,Article Display,"{'x': 0.550000011920929, 'y': 0.55000001192092...","{'x': -0.1365342140197754, 'y': -0.00066864676...",1st Conf,1st Conf,30.0
950,1652721932537,10-25-32-53,False,answer,4,Keypad,"{'x': 0.550000011920929, 'y': 0.55000001192092...","{'x': -0.3434654772281647, 'y': 0.026900703087...",9499274580,9499274580,30.0
1375,1652721958417,10-25-58-41,False,answer,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.39558348059654236, 'y': 0.04407547414...",8058272338,8058272338,30.0
1379,1652721984488,10-26-24-48,False,answer,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",76,6,30.0
2408,1652721985538,10-26-25-53,False,answer,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",63,63,30.0
2410,1652721985588,10-26-25-58,False,answer,7,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",63,,30.0


In [18]:

sessionsDf["Guesses"] = 0
sessionsDf["Time_To_First_Guess"] = 0


for i in guessEvents.index:
    userID = guessEvents["userID"][i]
    task_num = guessEvents["task_number"][i]
    event_time = guessEvents["unixTime"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Guesses"] += 1
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num) & (sessionsDf["Time_To_First_Guess"] == 0) & (sessionsDf["Actual_Start_Time"] < event_time)

    sessionsDf.loc[mask, "Time_To_First_Guess"] = event_time - sessionsDf["Actual_Start_Time"]


In [539]:
sessionsDf.loc[sessionsDf["Time_To_First_Guess"] < 0]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Time_To_First_Guess


In [19]:
sessionsDf.head(10)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Time_To_First_Guess
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792,1,2810
120,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165...",1652747526727,1652747540603,1,13876
240,53,153,The task is to click on the 2nd article title....,1653347136263,1653347142364,6101,0,True,"[{'userID': 53, 'frameNum': 1, 'unixTime': 165...",1653347136313,1653347142364,1,6051
330,41,80,The task is to click on the 2nd article title....,1652904601496,1652904604747,3251,0,True,"[{'userID': 41, 'frameNum': 1, 'unixTime': 165...",1652904601546,1652904604747,1,3201
388,71,110,The task is to click on the 2nd article title....,1653342438648,1653342442018,3370,0,True,"[{'userID': 71, 'frameNum': 1, 'unixTime': 165...",1653342438698,1653342442008,1,3310
110,60,103,The task is to click on the 2nd article title....,1653093121501,1653093124821,3320,0,False,"[{'userID': 60, 'frameNum': 1, 'unixTime': 165...",1653093121561,1653093124821,1,3260
380,85,87,The task is to click on the 2nd article title....,1653699238189,1653699241749,3560,0,True,"[{'userID': 85, 'frameNum': 1, 'unixTime': 165...",1653699238239,1653699241739,1,3500
40,32,117,The task is to click on the 2nd article title....,1652736971691,1652736975730,4039,0,False,"[{'userID': 32, 'frameNum': 1, 'unixTime': 165...",1652736971747,1652736975730,1,3983
260,37,52,The task is to click on the 2nd article title....,1652818005343,1652818008923,3580,0,True,"[{'userID': 37, 'frameNum': 1, 'unixTime': 165...",1652818005393,1652818008913,1,3520
270,39,220,The task is to click on the 2nd article title....,1652829254126,1652829262856,8730,0,True,"[{'userID': 39, 'frameNum': 1, 'unixTime': 165...",1652829254176,1652829262846,1,8670


In [20]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([           -50,            -60,            -50,            -50,
                  -50,            -60,            -50,            -56,
                  -50,            -50,            -50,            -61,
                  -50,            -50,            -60,            -56,
                  -50,            -61,            -60,            -51,
                  -60,            -50,            -49,            -52,
                  -62,            -60,            -59,            -50,
                  -60,            -49,            -51,            -61,
                  -50,            -50,            -50,            -50,
                  -43,            -60,            -60,            -45,
                  -60,            -50,            -60,            -51,
                  -60,            -45,            -40,            -30,
                  -40,            -40,            -43,            -40,
                  -50,            -40,            -38,            -39,
      

In [21]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Time_To_First_Guess,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792,1,2810,2810


In [22]:
sessionsDf.loc[sessionsDf["Time_To_First_Guess"] < 0]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Time_To_First_Guess,Actual_Total_Time


In [23]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([           59,            64,            50,            50,
                  60,            60,            60,            56,
                  60,            60,            66,            71,
                  67,            50,            70,            65,
                  60,            61,            70,            51,
                  66,            50,            49,            62,
                  66,            60,            59,            60,
                  70,            49,            53,            61,
                  60,            50,            60,            50,
                  43,            60,            70,            52,
                  70,            60,            72,            62,
                  60,            60,            50,            40,
                  51,            40,            43,            50,
                  50,            40,            38,            49,
                  40,            49,            50,           

In [24]:
allsessionsDf = sessionsDf.loc[sessionsDf['sessionNumber'] < 9]
allsessionsDf.to_excel("sessionsData.xlsx")

In [25]:
dataFrame = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Task', 'Guesses', 'Avg Dist', 'STD Dist', 'Avg Scale', 'Avg Scale:Dist', 'Avg Visual Angle', 'Std Visual Angle'])

ordered = sessionsDf.sort_values(by=["userID","sessionNumber"])
dataFrame['userID'] = ordered["userID"]
dataFrame['Task'] = ordered["sessionNumber"] + 1
dataFrame['Guesses'] = ordered["Guesses"]
dataFrame['Mode'] = ordered['isResponsive']
dataFrame['Time'] = ordered['Actual_Total_Time']
dataFrame['Time_To_First_Guess'] = ordered['Time_To_First_Guess']

dataFrame = dataFrame.loc[dataFrame["Task"] < 10]

#Removing weather tasks for user 56 and 85, were unable to complete them
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 7)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 8)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 9)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 7)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 8)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 9)]
dataFrame = dataFrame.drop(findRowToDrop.index)

In [26]:
sessionsDf.head(2)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Time_To_First_Guess,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792,1,2810,2810
120,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165...",1652747526727,1652747540603,1,13876,13876


In [27]:
dataFrame.loc[(dataFrame["userID"] == 31) & (dataFrame["Task"] == 1)]

,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Avg Visual Angle,Std Visual Angle,Time_To_First_Guess
210,31,True,NaN,7000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,7000


In [28]:
import math
import ast
def calcDistance(hPos, aPos):
    xDir = hPos['x'] - aPos['x']
    yDir = hPos['y'] - aPos['y']
    zDir = hPos['z'] - aPos['z']
    
    return math.sqrt(xDir*xDir + yDir*yDir + zDir*zDir)

for i in dataFrame.index:
    userID = dataFrame["userID"][i]
    task = dataFrame["Task"][i] - 1
    responsive = dataFrame['Mode'][i]
    findDistForSession = sessionsDf.loc[sessionsDf["userID"] == userID]

    startEvent = startEvents.loc[(startEvents["userID"] == userID) & (startEvents["task_number"] == task)]
    startTime = startEvent["unixTime"].to_numpy()[0]
    endEvent = endEvents.loc[(endEvents["userID"] == userID) & (endEvents["task_number"] == task)]
    endTime = endEvent["unixTime"].to_numpy()[0]
    framesUnpacked = list(itertools.chain(*findDistForSession["frames"]))
    sessionFramesDf = pandas.DataFrame(framesUnpacked)

    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] >= startTime]
    #sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= endTime + 100]
    sessionFramesDf = sessionFramesDf.sort_values("unixTime")

    lastFrame = sessionFramesDf.loc[sessionFramesDf["experimentEvents"].str.len() != 0]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] >= endTime]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] <= endTime + 600]

    if(lastFrame.shape[0] != 1):
        print(endTime)
        print(lastFrame.shape[0])
        print(lastFrame["unixTime"].to_numpy())

    lastFrameTime = lastFrame["unixTime"].to_numpy()[0]
    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= lastFrameTime]

    if(responsive):
        responsiveData = pandas.DataFrame(list(itertools.chain(sessionFramesDf["responsiveData"])))
        dist_avg = responsiveData['distance'].mean()
        dist_std = responsiveData['distance'].std()
        scales = pandas.DataFrame(list(itertools.chain(responsiveData["scale"])))
        for j in responsiveData.index:
            mask = j == responsiveData.index
            dist = responsiveData.loc[mask, 'distance']
            scale = scales.loc[mask, 'y']
            responsiveData.loc[mask, 'VA'] = math.atan(scale/dist)
        scales = scales.loc[scales['y'] != 0]
        scale_avg = scales['y'].mean()
        
        va_avg = responsiveData['VA'].mean()
        va_std = responsiveData['VA'].std()

    else:
        Positions = pandas.DataFrame(columns=['hPos', 'aPos', 'unixTime', 'distance', 'scale', 'VA'])
        Positions["hPos"] = sessionFramesDf["hPos"]
        Positions["unixTime"] = sessionFramesDf["unixTime"]
        experimentEventsForTask = pandas.DataFrame(list(itertools.chain(*sessionFramesDf["experimentEvents"])))
        experimentEventsForTask = experimentEventsForTask.loc[experimentEventsForTask["task_number"] == task]
        for t in experimentEventsForTask.index:
            event_time = experimentEventsForTask["unixTime"][t]
            mask = (sessionFramesDf['unixTime'] >= event_time)
            objectScale = {}
            objectPos = {}
            objectPos = experimentEventsForTask["object_position"][t]
            objectScale = experimentEventsForTask["object_scale"][t]
            Positions.loc[mask, 'aPos'] = str(objectPos)
            Positions.loc[mask, 'scale'] = objectScale['y']
            

        for j in Positions.index:
            hPos = Positions["hPos"][j]
            aPos = Positions["aPos"][j]
            aPos = ast.literal_eval(aPos)
            mask = Positions.index == j
            distance = calcDistance(hPos, aPos)
            Positions.loc[mask, 'distance'] = distance
            Positions.loc[mask,'VA'] = math.atan(Positions.loc[mask, 'scale']/distance)


        dist_avg = Positions['distance'].mean()
        dist_std = Positions['distance'].std()
        scale_avg = Positions['scale'].mean()

        va_avg = Positions['VA'].mean()
        va_std = Positions['VA'].std()

    mask = dataFrame.index == i
    dataFrame.loc[mask, 'Avg Dist'] = dist_avg
    dataFrame.loc[mask, 'STD Dist'] = dist_std
    dataFrame.loc[mask, 'Avg Scale'] = scale_avg
    dataFrame.loc[mask, 'Avg Scale:Dist'] = scale_avg/dist_avg
    dataFrame.loc[mask, 'Avg Visual Angle'] = va_avg
    dataFrame.loc[mask, 'Std Visual Angle'] = va_std

dataFrame.loc[(dataFrame["userID"] == 31) & (dataFrame["Task"] == 1)]

1653088273395
2
[1653088273475 1653088273736]


,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Avg Visual Angle,Std Visual Angle,Time_To_First_Guess
210,31,True,NaN,7000,1,1,0.649374,0.019955,0.25,0.384986,0.367769,0.010313,7000


In [29]:
dataFrame.head(12)

,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Avg Visual Angle,Std Visual Angle,Time_To_First_Guess
250,30,False,NaN,17181,1,1,0.650775,0.026776,0.25,0.384157,0.36726,0.013621,17181
251,30,False,NaN,12720,2,1,0.606882,0.011124,0.25,0.411942,0.39086,0.00656,12720
252,30,False,NaN,17170,3,1,0.622351,0.055196,0.35,0.562383,0.514907,0.039313,17170
253,30,False,NaN,9960,4,1,0.556536,0.117192,0.55,0.988256,0.789284,0.08949,9960
254,30,False,NaN,21691,5,1,0.456405,0.14284,0.55,1.205071,0.893916,0.113065,21691
255,30,False,NaN,16270,6,1,0.522625,0.197677,0.9,1.722076,1.058903,0.128981,16270
256,30,False,NaN,16911,7,2,0.563273,0.253812,0.3,0.532602,0.539647,0.154948,15861
257,30,False,NaN,41520,8,21,0.565801,0.148868,0.43,0.759984,0.662565,0.072144,9390
258,30,False,NaN,9360,9,3,0.76461,0.302052,0.77,1.00705,0.820143,0.153506,8430
210,31,True,NaN,7000,1,1,0.649374,0.019955,0.25,0.384986,0.367769,0.010313,7000


In [30]:
#converting from task number to application and sub task number

mask = dataFrame['Task'] <= 4
dataFrame.loc[mask, 'Application'] = "Article"

mask = dataFrame['Task'] == 5
dataFrame.loc[mask, 'Task'] = 1
dataFrame.loc[mask, 'Application'] = "Numpad"

mask = dataFrame['Task'] == 6
dataFrame.loc[mask, 'Task'] = 2

dataFrame.loc[mask, 'Application'] = "Numpad"

mask = dataFrame['Task'] == 7
dataFrame.loc[mask, 'Task'] = 1
dataFrame.loc[mask, 'Application'] = "Weather"

mask = dataFrame['Task'] == 8
dataFrame.loc[mask, 'Task'] = 2
dataFrame.loc[mask, 'Application'] = "Weather"

mask = dataFrame['Task'] == 9
dataFrame.loc[mask, 'Task'] = 3

dataFrame.loc[mask, 'Application'] = "Weather"

In [31]:
mask = dataFrame['Mode'] == False
dataFrame.loc[mask, 'Mode'] = "Static"

mask = dataFrame['Mode'] == True
dataFrame.loc[mask, 'Mode'] = "Responsive"

dataFrame

,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Avg Visual Angle,Std Visual Angle,Time_To_First_Guess
250,30,Static,Article,17181,1,1,0.650775,0.026776,0.25,0.384157,0.36726,0.013621,17181
251,30,Static,Article,12720,2,1,0.606882,0.011124,0.25,0.411942,0.39086,0.00656,12720
252,30,Static,Article,17170,3,1,0.622351,0.055196,0.35,0.562383,0.514907,0.039313,17170
253,30,Static,Article,9960,4,1,0.556536,0.117192,0.55,0.988256,0.789284,0.08949,9960
254,30,Static,Numpad,21691,1,1,0.456405,0.14284,0.55,1.205071,0.893916,0.113065,21691
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,86,Static,Numpad,47231,1,1,0.558575,0.06417,0.55,0.984649,0.781088,0.059844,47231
165,86,Static,Numpad,23550,2,1,0.576774,0.015385,0.9,1.560403,1.000967,0.012062,23550
166,86,Static,Weather,44681,1,2,0.593761,0.09708,0.3,0.505254,0.477302,0.074278,43471
167,86,Static,Weather,30271,2,11,0.671105,0.01976,0.43,0.640734,0.570109,0.013207,14551


In [32]:
dataFrame = dataFrame.sort_values(by= ['Application', 'Task', 'Mode', 'Time'])
dataFrame.to_csv("ExperimentData.csv")

articleDf = dataFrame.loc[dataFrame["Application"] == "Article"]
articleDf = articleDf.sort_values(by= ['Task', 'Mode', 'Time'])
articleDf.to_csv("ArticleTasksExperimentData.csv")
numpadDf = dataFrame.loc[dataFrame["Application"] == "Numpad"]
numpadDf = numpadDf.sort_values(by= ['Task', 'Mode', 'Time'])
numpadDf.to_csv("NumpadTasksExperimentData.csv")
weatherDf = dataFrame.loc[dataFrame["Application"] == "Weather"]
weatherDf = weatherDf.sort_values(by= ['Task', 'Mode', 'Time'])
weatherDf.to_csv("WeatherTasksExperimentData.csv")

import math

In [554]:
'''
for i in startEvents.index:
    user_id = startEvents["userID"][i]
    task = startEvents["task_number"][i]
    endEvent = endEvents.loc[(endEvents["userID"] == user_id) & (endEvents["task_number"] == task)]
    initialScale = startEvents["object_scale"][i]
    initialPos = startEvents["object_position"][i]
    endScale = endEvent["object_scale"].to_numpy()[0]
    endPos = endEvent["object_position"].to_numpy()[0]
    if(initialScale != endScale):
        print(f"scale before {initialScale}")
        print(f"scale after {endScale}")
        print(user_id)
        print(task)
        print(endEvent["isResponsive"])

    if(initialPos != endPos):
        print(f"pos before {initialPos}")
        print(f"pos after {endPos}")
        print(user_id)
        print(task)
        print(endEvent["isResponsive"])
'''

'\nfor i in startEvents.index:\n    user_id = startEvents["userID"][i]\n    task = startEvents["task_number"][i]\n    endEvent = endEvents.loc[(endEvents["userID"] == user_id) & (endEvents["task_number"] == task)]\n    initialScale = startEvents["object_scale"][i]\n    initialPos = startEvents["object_position"][i]\n    endScale = endEvent["object_scale"].to_numpy()[0]\n    endPos = endEvent["object_position"].to_numpy()[0]\n    if(initialScale != endScale):\n        print(f"scale before {initialScale}")\n        print(f"scale after {endScale}")\n        print(user_id)\n        print(task)\n        print(endEvent["isResponsive"])\n\n    if(initialPos != endPos):\n        print(f"pos before {initialPos}")\n        print(f"pos after {endPos}")\n        print(user_id)\n        print(task)\n        print(endEvent["isResponsive"])\n'

In [33]:
article =  dataFrame.loc[dataFrame["Application"] == "Article"]
for i in range(1,5):
    taskDf = article.loc[article["Task"] == i]
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"ARTICLE TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))
    print(scipy.stats.mannwhitneyu(task_r['Time'], task_n['Time']))
    print(scipy.stats.mannwhitneyu(task_r['Guesses'], task_n['Guesses']))



ARTICLE TASK 1: 
responsive task 1 Time: 5868.522 ± 2744.399
normal task 1 Time: 8721.682 ± 9498.250
Ttest_indResult(statistic=-1.3821953988748257, pvalue=0.17404979230459425)
responsive task 1 Guesses: 1.043 ± 0.209
normal task 1 Guesses: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
responsive task 1 Dist: 0.611 ± 0.043
normal task 1 Dist: 0.624 ± 0.044
Ttest_indResult(statistic=-0.9907393823025629, pvalue=0.32735539311446293)
MannwhitneyuResult(statistic=222.5, pvalue=0.24787377921503706)
MannwhitneyuResult(statistic=241.0, pvalue=0.2728493648811271)
ARTICLE TASK 2: 
responsive task 2 Time: 5195.391 ± 3871.828
normal task 2 Time: 13861.909 ± 12837.685
Ttest_indResult(statistic=-3.0951827958607496, pvalue=0.003453951553728935)
responsive task 2 Guesses: 1.130 ± 0.458
normal task 2 Guesses: 1.500 ± 0.859
Ttest_indResult(statistic=-1.8121843206506378, pvalue=0.0769443111920115)
responsive task 2 Dist: 0.617 ± 0.061
normal task 2 Dist: 0.599 ± 0.

In [34]:
article.head(5)

,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Avg Visual Angle,Std Visual Angle,Time_To_First_Guess
0,61,Responsive,Article,2810,1,1,0.601711,0.008941,0.25,0.415482,0.393846,0.005281,2810
100,67,Responsive,Article,3040,1,1,0.550849,0.047947,0.25,0.453845,0.428482,0.035151,3040
330,41,Responsive,Article,3201,1,1,0.582287,0.106483,0.25,0.429341,0.394199,0.004151,3201
388,71,Responsive,Article,3310,1,1,0.643874,0.045446,0.25,0.388274,0.371827,0.024781,3310
380,85,Responsive,Article,3500,1,1,0.61662,0.014638,0.25,0.405436,0.38535,0.008432,3500


In [35]:

articleTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio', 'Std Dist', 'Avg Visual Angle', 'Std Visual Angle' ])
articleTasks["userID"] = article["userID"]
articleTasks["Mode"] = article["Mode"]
articleTasks = articleTasks.drop_duplicates(subset=["userID"], keep="first")
articleTasks = articleTasks.sort_values(by='Time')

for i in articleTasks.index:
    userID = articleTasks["userID"][i]
    total_time = 0
    user = article.loc[article["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])

    mask = articleTasks.index == i

    articleTasks.loc[mask, "Time"] = total_time
    articleTasks.loc[mask,"Guesses"] = total_guesses
    articleTasks.loc[mask,"Application"] = "Article"
    articleTasks.loc[mask,"Avg Dist"] = sum(user["Avg Dist"])/4
    articleTasks.loc[mask,"Avg Ratio"] = sum(user["Avg Scale:Dist"])/4
    articleTasks.loc[mask,"Std Dist"] = sum(user["STD Dist"])/4
    articleTasks.loc[mask,"Avg Visual Angle"] = sum(user['Avg Visual Angle'])/4
    articleTasks.loc[mask,"Std Visual Angle"] = sum(user['Std Visual Angle'])/4

'''
for i in articleTasks.index:
    userID = articleTasks["userID"][i]
    user = article.loc[article["userID"] == userID]
    total_time = articleTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    std_dist = 0
    std_angle = 0
    avg_angle = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        std_dist +=  user["STD Dist"][j]
        avg_dist += user["Avg Dist"][j]
        avg_ratio += user["Avg Scale:Dist"][j]
        avg_angle += user["Avg Visual Angle"][j]
        std_angle += user["Std Visual Angle"][j]

    articleTasks["Avg Dist"][i] = avg_dist
    articleTasks["Avg Ratio"][i] = avg_ratio
    articleTasks["Std Dist"][i] = std_dist
    articleTasks["Avg Visual Angle"] = avg_angle
    articleTasks["Std Visual Angle"] = std_angle
'''

articleTasks = articleTasks.sort_values(by= ['Mode', 'Time'])
articleTasks.to_csv("ArticleExperimentData.csv")
articleTasks.head(5)

,userID,Mode,Application,Time,Guesses,Avg Dist,Avg Ratio,Std Dist,Avg Visual Angle,Std Visual Angle
0,61,Responsive,Article,10803,4,0.607538,0.571814,0.015383,0.507244,0.019267
260,37,Responsive,Article,15321,5,0.617966,0.5657,0.00912,0.502892,0.009149
210,31,Responsive,Article,17031,4,0.634971,0.550685,0.02194,0.492336,0.017657
130,33,Responsive,Article,18449,4,0.645107,0.542025,0.069303,0.491987,0.050237
408,49,Responsive,Article,18892,6,0.638577,0.545923,0.043143,0.489089,0.019817


In [36]:
taskDf = articleTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]
print(f"ARTICLE TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")
print(scipy.stats.describe(task_r['Time']))

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")
print(scipy.stats.describe(task_n['Time']))

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
print(scipy.stats.shapiro(task_r['Time']))
print(scipy.stats.shapiro(task_n['Time']))
print(scipy.stats.mannwhitneyu(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))
print(scipy.stats.shapiro(task_r['Guesses']))
print(scipy.stats.shapiro(task_n['Guesses']))
print(scipy.stats.mannwhitneyu(task_r['Guesses'], task_n['Guesses']))


taskRAvgTime = task_r['Avg Dist'].mean()
taskRStd = task_r['Avg Dist'].std()
print(f"responsive tasks Avg Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))
print(scipy.stats.shapiro(task_r['Avg Dist']))
print(scipy.stats.shapiro(task_n['Avg Dist']))
print(scipy.stats.mannwhitneyu(task_r['Avg Dist'], task_n['Avg Dist']))

taskRAvgTime = task_r['Std Dist'].mean()
taskRStd = task_r['Std Dist'].std()
print(f"responsive tasks Std Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Std Dist'].mean()
taskNStd = task_n['Std Dist'].std()
print(f"normal tasks Std Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Std Dist'], task_n['Std Dist']))
print(scipy.stats.shapiro(task_r['Std Dist']))
print(scipy.stats.shapiro(task_n['Std Dist']))
print(scipy.stats.mannwhitneyu(task_r['Std Dist'], task_n['Std Dist']))


ARTICLE TASKS: 
responsive tasks Time: 35496.000 ± 22396.322
DescribeResult(nobs=23, minmax=(10803, 108861), mean=35496.0, variance=501595240.6363636, skewness=1.70547911345168, kurtosis=2.993294329859765)
normal tasks Time: 45099.091 ± 25403.666
DescribeResult(nobs=22, minmax=(18383, 128797), mean=45099.09090909091, variance=645346233.2294372, skewness=1.8446390132543211, kurtosis=3.5774806439565676)
Ttest_indResult(statistic=-1.346660943843676, pvalue=0.18514493767770876)
ShapiroResult(statistic=0.8201521039009094, pvalue=0.0008199320873245597)
ShapiroResult(statistic=0.8093016147613525, pvalue=0.0007021405035629869)
MannwhitneyuResult(statistic=178.0, pvalue=0.04536297444450874)
responsive tasks Guesses: 5.130 ± 1.792
normal tasks Guesses: 6.318 ± 2.297
Ttest_indResult(statistic=-1.939236524146663, pvalue=0.05905204123368115)
ShapiroResult(statistic=0.6491820812225342, pvalue=3.2814816677273484e-06)
ShapiroResult(statistic=0.8727595210075378, pvalue=0.008814198896288872)
Mannwhitney

In [37]:
taskRAvgTime = task_r['Std Visual Angle'].mean()
taskRStd = task_r['Std Visual Angle'].std()
print(f"responsive tasks Std Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Std Visual Angle'].mean()
taskNStd = task_n['Std Visual Angle'].std()
print(f"normal tasks Std Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Std Visual Angle'], task_n['Std Visual Angle']))
print(scipy.stats.shapiro(task_r['Std Visual Angle']))
print(scipy.stats.shapiro(task_n['Std Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Std Visual Angle'], task_n['Std Visual Angle']))

taskRAvgTime = task_r['Avg Visual Angle'].mean()
taskRStd = task_r['Avg Visual Angle'].std()
print(f"responsive tasks Avg Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Visual Angle'].mean()
taskNStd = task_n['Avg Visual Angle'].std()
print(f"normal tasks Avg Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))
print(scipy.stats.shapiro(task_r['Avg Visual Angle']))
print(scipy.stats.shapiro(task_n['Avg Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))

responsive tasks Std Visual Angle: 0.034 ± 0.021
normal tasks Std Visual Angle: 0.031 ± 0.024
Ttest_indResult(statistic=0.38271943943044007, pvalue=0.7038131558156246)
ShapiroResult(statistic=0.8670645952224731, pvalue=0.0056366464123129845)
ShapiroResult(statistic=0.7817897200584412, pvalue=0.0002631171082612127)
MannwhitneyuResult(statistic=226.0, pvalue=0.2736858590746263)
responsive tasks Avg Visual Angle: 0.501 ± 0.021
normal tasks Avg Visual Angle: 0.509 ± 0.021
Ttest_indResult(statistic=-1.2754698892372287, pvalue=0.20899080817876337)
ShapiroResult(statistic=0.9567854404449463, pvalue=0.40162545442581177)
ShapiroResult(statistic=0.9665436148643494, pvalue=0.6313179731369019)
MannwhitneyuResult(statistic=198.0, pvalue=0.1079572360837684)


In [38]:
numpad =  dataFrame.loc[dataFrame["Application"] == "Numpad"]
for i in range(1,3):
    taskDf = numpad.loc[numpad["Task"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"Numpad TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    print(scipy.stats.mannwhitneyu(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))
    print(scipy.stats.mannwhitneyu(task_r['Guesses'], task_n['Guesses']))

'''
    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))
'''
    



Numpad TASK 1: 
responsive task 1 Time: 28847.609 ± 13268.076
normal task 1 Time: 24911.773 ± 9271.263
Ttest_indResult(statistic=1.1485301906892023, pvalue=0.25709893555650787)
MannwhitneyuResult(statistic=205.0, pvalue=0.1403998531200749)
responsive task 1 Guesses: 1.043 ± 0.209
normal task 1 Guesses: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
MannwhitneyuResult(statistic=241.0, pvalue=0.2728493648811271)
Numpad TASK 2: 
responsive task 2 Time: 53535.391 ± 59750.494
normal task 2 Time: 31114.227 ± 22548.576
Ttest_indResult(statistic=1.650557169528861, pvalue=0.10611550592249368)
MannwhitneyuResult(statistic=185.0, pvalue=0.06268226149150617)
responsive task 2 Guesses: 1.261 ± 0.752
normal task 2 Guesses: 1.045 ± 0.213
Ttest_indResult(statistic=1.2944933360701294, pvalue=0.2024037595750517)
MannwhitneyuResult(statistic=230.5, pvalue=0.15578443706704742)


'\n    taskRAvgTime = task_r[\'Avg Dist\'].mean()\n    taskRStd = task_r[\'Avg Dist\'].std()\n    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")\n\n    taskNAvgTime = task_n[\'Avg Dist\'].mean()\n    taskNStd = task_n[\'Avg Dist\'].std()\n    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")\n\n    print(scipy.stats.ttest_ind(task_r[\'Avg Dist\'], task_n[\'Avg Dist\']))\n'

In [39]:
numpadTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio', 'Avg Visual Angle', 'Std Visual Angle'])
numpadTasks["userID"] = numpad["userID"]
numpadTasks["Mode"] = numpad["Mode"]
numpadTasks = numpadTasks.drop_duplicates(subset=["userID"], keep="first")

for i in numpadTasks.index:
    userID = numpadTasks["userID"][i]
    total_time = 0
    user = numpad.loc[numpad["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])

    mask = i == numpadTasks.index

    numpadTasks.loc[mask,"Time"] = total_time
    numpadTasks.loc[mask,"Guesses"] = total_guesses
    numpadTasks.loc[mask,"Application"] = "Numpad"

    numpadTasks.loc[mask,"Avg Dist"] = sum(user["Avg Dist"])/2
    numpadTasks.loc[mask,"Avg Ratio"] = sum(user["Avg Scale:Dist"])/2
    numpadTasks.loc[mask,"Std Dist"] = sum(user["STD Dist"])/2
    numpadTasks.loc[mask,"Avg Visual Angle"] = sum(user['Avg Visual Angle'])/2
    numpadTasks.loc[mask,"Std Visual Angle"] = sum(user['Std Visual Angle'])/2

'''
for i in numpadTasks.index:
    userID = numpadTasks["userID"][i]
    user = numpad.loc[numpad["userID"] == userID]
    total_time = numpadTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        avg_dist += weight * user["Avg Dist"][j]
        avg_ratio += weight * user["Avg Scale:Dist"][j]

    numpadTasks["Avg Dist"][i] = avg_dist
    numpadTasks["Avg Ratio"][i] = avg_ratio
'''
numpadTasks = numpadTasks.sort_values(by= ['Mode', 'Time'])
numpadTasks.to_csv("NumpadExperimentData.csv")

taskDf = numpadTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]


In [40]:
print(f"NUMPAD TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
print(scipy.stats.shapiro(task_r['Time']))
print(scipy.stats.shapiro(task_n['Time']))
print(scipy.stats.mannwhitneyu(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))
print(scipy.stats.shapiro(task_r['Guesses']))
print(scipy.stats.shapiro(task_n['Guesses']))
print(scipy.stats.mannwhitneyu(task_r['Guesses'], task_n['Guesses']))


taskNAvgTime = task_r['Avg Dist'].mean()
taskNStd = task_r['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.shapiro(task_r['Avg Dist']))
print(scipy.stats.shapiro(task_n['Avg Dist']))
print(scipy.stats.mannwhitneyu(task_r['Avg Dist'], task_n['Avg Dist']))
print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

taskNAvgTime = task_r['Std Dist'].mean()
taskNStd = task_r['Std Dist'].std()
print(f"normal tasks Std Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

taskNAvgTime = task_n['Std Dist'].mean()
taskNStd = task_n['Std Dist'].std()
print(f"normal tasks Std Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.shapiro(task_r['Std Dist']))
print(scipy.stats.shapiro(task_n['Std Dist']))
print(scipy.stats.mannwhitneyu(task_r['Std Dist'], task_n['Std Dist']))
print(scipy.stats.ttest_ind(task_r['Std Dist'], task_n['Std Dist']))

taskRAvgTime = task_r['Avg Ratio'].mean()
taskRStd = task_r['Avg Ratio'].std()
print(f"responsive tasks Avg Ratio: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Ratio'].mean()
taskNStd = task_n['Avg Ratio'].std()
print(f"normal tasks Avg Ratio: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Ratio'], task_n['Avg Ratio']))




NUMPAD TASKS: 
responsive tasks Time: 82383.000 ± 59884.352
normal tasks Time: 56026.000 ± 22913.590
Ttest_indResult(statistic=1.9327222234179222, pvalue=0.059873907233486755)
ShapiroResult(statistic=0.7168604731559753, pvalue=2.3349291950580664e-05)
ShapiroResult(statistic=0.8527014851570129, pvalue=0.0037908463273197412)
MannwhitneyuResult(statistic=180.0, pvalue=0.04986447066462151)
responsive tasks Guesses: 2.304 ± 0.765
normal tasks Guesses: 2.136 ± 0.351
Ttest_indResult(statistic=0.9394209291244621, pvalue=0.35276202948893265)
ShapiroResult(statistic=0.47010505199432373, pvalue=4.495635863577263e-08)
ShapiroResult(statistic=0.4117925763130188, pvalue=2.1539008798754367e-08)
MannwhitneyuResult(statistic=240.5, pvalue=0.33264084183130876)
normal tasks Avg Dist: 0.561 ± 0.059
normal tasks Avg Dist: 0.559 ± 0.064
ShapiroResult(statistic=0.945949912071228, pvalue=0.24068719148635864)
ShapiroResult(statistic=0.9296372532844543, pvalue=0.12058009207248688)
MannwhitneyuResult(statistic=2

In [41]:
taskRAvgTime = task_r['Std Visual Angle'].mean()
taskRStd = task_r['Std Visual Angle'].std()
print(f"responsive tasks Std Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Std Visual Angle'].mean()
taskNStd = task_n['Std Visual Angle'].std()
print(f"normal tasks Std Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Std Visual Angle'], task_n['Std Visual Angle']))
print(scipy.stats.shapiro(task_r['Std Visual Angle']))
print(scipy.stats.shapiro(task_n['Std Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Std Visual Angle'], task_n['Std Visual Angle']))

taskRAvgTime = task_r['Avg Visual Angle'].mean()
taskRStd = task_r['Avg Visual Angle'].std()
print(f"responsive tasks Avg Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Visual Angle'].mean()
taskNStd = task_n['Avg Visual Angle'].std()
print(f"normal tasks Avg Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))
print(scipy.stats.shapiro(task_r['Avg Visual Angle']))
print(scipy.stats.shapiro(task_n['Avg Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))

responsive tasks Std Visual Angle: 0.076 ± 0.039
normal tasks Std Visual Angle: 0.059 ± 0.028
Ttest_indResult(statistic=1.671837822634168, pvalue=0.10181899200110935)
ShapiroResult(statistic=0.9226189255714417, pvalue=0.07582003623247147)
ShapiroResult(statistic=0.9180294275283813, pvalue=0.06920820474624634)
MannwhitneyuResult(statistic=181.0, pvalue=0.05224519524155745)
responsive tasks Avg Visual Angle: 0.906 ± 0.045
normal tasks Avg Visual Angle: 0.905 ± 0.050
Ttest_indResult(statistic=0.08800233844729619, pvalue=0.9302833882543262)
ShapiroResult(statistic=0.9808107018470764, pvalue=0.9194939136505127)
ShapiroResult(statistic=0.9804049730300903, pvalue=0.9222099184989929)
MannwhitneyuResult(statistic=251.0, pvalue=0.48641518223101404)


In [42]:
weather =  dataFrame.loc[dataFrame["Application"] == "Weather"]
for i in range(1,4):
    taskDf = weather.loc[weather["Task"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"Weather TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

    '''

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))


    taskRAvgTime = task_r['Time_To_First_Guess'].mean()
    taskRStd = task_r['Time_To_First_Guess'].std()
    print(f"responsive task {str(i)} TT1G: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time_To_First_Guess'].mean()
    taskNStd = task_n['Time_To_First_Guess'].std()
    print(f"normal task {str(i)} TT1G: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.shapiro(task_r['Time_To_First_Guess']))
    print(scipy.stats.shapiro(task_n['Time_To_First_Guess']))

    print(scipy.stats.ttest_ind(task_r['Time_To_First_Guess'], task_n['Time_To_First_Guess']))
    print(scipy.stats.mannwhitneyu(task_r['Time_To_First_Guess'], task_n['Time_To_First_Guess']))

    '''

Weather TASK 1: 
responsive task 1 Time: 56057.636 ± 91488.185
normal task 1 Time: 26658.952 ± 13485.498
Ttest_indResult(statistic=1.4567480367843535, pvalue=0.1528023569915703)
responsive task 1 Guesses: 7.955 ± 12.053
normal task 1 Guesses: 5.048 ± 5.491
Ttest_indResult(statistic=1.009331866898274, pvalue=0.31873535675680703)
responsive task 1 Dist: 0.697 ± 0.102
normal task 1 Dist: 0.577 ± 0.045
Ttest_indResult(statistic=4.947774806711382, pvalue=1.32661962396392e-05)
Weather TASK 2: 
responsive task 2 Time: 59949.182 ± 30794.566
normal task 2 Time: 57360.238 ± 68450.821
Ttest_indResult(statistic=0.1611999509384577, pvalue=0.8727276363332076)
responsive task 2 Guesses: 22.500 ± 8.755
normal task 2 Guesses: 22.286 ± 12.207
Ttest_indResult(statistic=0.06638587784263188, pvalue=0.947393280284176)
responsive task 2 Dist: 0.600 ± 0.058
normal task 2 Dist: 0.592 ± 0.050
Ttest_indResult(statistic=0.5135610520937607, pvalue=0.6103155668620123)
Weather TASK 3: 
responsive task 3 Time: 49404.

In [43]:
weatherTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio', 'Time_To_First_Guess', 'Std Dist', 'Avg Visual Angle', "Std Visual Angle" ])
weatherTasks["userID"] = weather["userID"]
weatherTasks["Mode"] = weather["Mode"]
weatherTasks = weatherTasks.drop_duplicates(subset=["userID"], keep="first")

for i in weatherTasks.index:
    userID = weatherTasks["userID"][i]
    total_time = 0
    user = weather.loc[weather["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])
    first_guess_time = sum(user["Time_To_First_Guess"])

    mask = i == weatherTasks.index
    weatherTasks.loc[mask, "Application"] = "Weather"
    weatherTasks.loc[mask,"Time"] = total_time
    weatherTasks.loc[mask,"Guesses"] = total_guesses
    weatherTasks.loc[mask,"Application"] = "Numpad"

    weatherTasks.loc[mask,"Avg Dist"] = sum(user["Avg Dist"])/3
    weatherTasks.loc[mask,"Avg Ratio"] = sum(user["Avg Scale:Dist"])/3
    weatherTasks.loc[mask,"Std Dist"] = sum(user["STD Dist"])/3
    weatherTasks.loc[mask,"Avg Visual Angle"] = sum(user['Avg Visual Angle'])/3
    weatherTasks.loc[mask,"Std Visual Angle"] = sum(user['Std Visual Angle'])/3

'''
for i in weatherTasks.index:
    userID = weatherTasks["userID"][i]
    user = weather.loc[weather["userID"] == userID]
    total_time = weatherTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    std_dist = 0
    avg_va = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        std_dist +=  user["STD Dist"][j]
        avg_dist +=  user["Avg Dist"][j]
        avg_ratio +=  user["Avg Scale:Dist"][j]
        avg_va +=  user["Avg Visual Angle"][j]

    weatherTasks["Avg Dist"][i] = avg_dist/3
    weatherTasks["Avg Ratio"][i] = avg_ratio/3
    weatherTasks["Std Dist"][i] = std_dist/3
    weatherTasks["Avg Visual Angle"][i] = avg_va/3

'''

weatherTasks = weatherTasks.sort_values(by= ['Mode', 'Time'])
weatherTasks.to_csv("WeatherExperimentData.csv")


In [44]:
taskDf = weatherTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]

print(f"WEATHER TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

print(scipy.stats.shapiro(task_r['Time']))
print(scipy.stats.shapiro(task_n['Time']))
print(scipy.stats.shapiro(task_r['Guesses']))
print(scipy.stats.shapiro(task_n['Guesses']))

print(task_r['Time'].shape[0])
print(task_n['Time'].shape[0])
print(scipy.stats.mannwhitneyu(task_r['Time'], task_n['Time']))
print(scipy.stats.mannwhitneyu(task_r['Guesses'], task_n['Guesses']))

taskRAvgTime = task_r['Avg Dist'].mean()
taskRStd = task_r['Avg Dist'].std()
print(f"responsive tasks Avg Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))
print(scipy.stats.shapiro(task_r['Avg Dist']))
print(scipy.stats.shapiro(task_n['Avg Dist']))
print(scipy.stats.mannwhitneyu(task_r['Avg Dist'], task_n['Avg Dist']))

taskRAvgTime = task_r['Std Dist'].mean()
taskRStd = task_r['Std Dist'].std()
print(f"responsive tasks Std Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Std Dist'].mean()
taskNStd = task_n['Std Dist'].std()
print(f"normal tasks Std Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Std Dist'], task_n['Std Dist']))
print(scipy.stats.shapiro(task_r['Std Dist']))
print(scipy.stats.shapiro(task_n['Std Dist']))
print(scipy.stats.mannwhitneyu(task_r['Std Dist'], task_n['Std Dist']))


taskRAvgTime = task_r['Avg Visual Angle'].mean()
taskRStd = task_r['Avg Visual Angle'].std()
print(f"responsive tasks Avg Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Visual Angle'].mean()
taskNStd = task_n['Avg Visual Angle'].std()
print(f"normal tasks Avg Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))
print(scipy.stats.shapiro(task_r['Avg Visual Angle']))
print(scipy.stats.shapiro(task_n['Avg Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))

'''
taskRAvgTime = task_r['Time_To_First_Guess'].mean()
taskRStd = task_r['Time_To_First_Guess'].std()
print(f"responsive task {str(i)} TT1G: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time_To_First_Guess'].mean()
taskNStd = task_n['Time_To_First_Guess'].std()
print(f"normal task {str(i)} TT1G: {taskNAvgTime:.3f} ± {taskNStd:.3f}")


print(scipy.stats.shapiro(task_r['Time_To_First_Guess']))
print(scipy.stats.shapiro(task_n['Time_To_First_Guess']))

print(scipy.stats.ttest_ind(task_r['Time_To_First_Guess'], task_n['Time_To_First_Guess']))
print(scipy.stats.mannwhitneyu(task_r['Time_To_First_Guess'], task_n['Time_To_First_Guess']))
'''


WEATHER TASKS: 
responsive tasks Time: 165411.727 ± 105099.860
normal tasks Time: 127929.810 ± 84347.194
Ttest_indResult(statistic=1.285928831791032, pvalue=0.2056831687677855)
responsive tasks Guesses: 44.682 ± 27.753
normal tasks Guesses: 38.905 ± 17.254
Ttest_indResult(statistic=0.815095492541208, pvalue=0.41972743373007315)
ShapiroResult(statistic=0.8304761648178101, pvalue=0.0015627911780029535)
ShapiroResult(statistic=0.8355544209480286, pvalue=0.002419202122837305)
ShapiroResult(statistic=0.7347895503044128, pvalue=5.6100845540640876e-05)
ShapiroResult(statistic=0.908470630645752, pvalue=0.05122752860188484)
22
21
MannwhitneyuResult(statistic=169.0, pvalue=0.06755732494788166)
MannwhitneyuResult(statistic=216.0, pvalue=0.3619415681477446)
responsive tasks Avg Dist: 0.644 ± 0.049
normal tasks Avg Dist: 0.614 ± 0.042
Ttest_indResult(statistic=2.144789219416111, pvalue=0.037944682982414435)
ShapiroResult(statistic=0.9761867523193359, pvalue=0.8472632765769958)
ShapiroResult(statist

'\ntaskRAvgTime = task_r[\'Time_To_First_Guess\'].mean()\ntaskRStd = task_r[\'Time_To_First_Guess\'].std()\nprint(f"responsive task {str(i)} TT1G: {taskRAvgTime:.3f} ± {taskRStd:.3f}")\n\ntaskNAvgTime = task_n[\'Time_To_First_Guess\'].mean()\ntaskNStd = task_n[\'Time_To_First_Guess\'].std()\nprint(f"normal task {str(i)} TT1G: {taskNAvgTime:.3f} ± {taskNStd:.3f}")\n\n\nprint(scipy.stats.shapiro(task_r[\'Time_To_First_Guess\']))\nprint(scipy.stats.shapiro(task_n[\'Time_To_First_Guess\']))\n\nprint(scipy.stats.ttest_ind(task_r[\'Time_To_First_Guess\'], task_n[\'Time_To_First_Guess\']))\nprint(scipy.stats.mannwhitneyu(task_r[\'Time_To_First_Guess\'], task_n[\'Time_To_First_Guess\']))\n'

In [45]:
taskRAvgTime = task_r['Std Visual Angle'].mean()
taskRStd = task_r['Std Visual Angle'].std()
print(f"responsive tasks Std Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Std Visual Angle'].mean()
taskNStd = task_n['Std Visual Angle'].std()
print(f"normal tasks Std Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Std Visual Angle'], task_n['Std Visual Angle']))
print(scipy.stats.shapiro(task_r['Std Visual Angle']))
print(scipy.stats.shapiro(task_n['Std Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Std Visual Angle'], task_n['Std Visual Angle']))

taskRAvgTime = task_r['Avg Visual Angle'].mean()
taskRStd = task_r['Avg Visual Angle'].std()
print(f"responsive tasks Avg Visual Angle: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Visual Angle'].mean()
taskNStd = task_n['Avg Visual Angle'].std()
print(f"normal tasks Avg Visual Angle: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))
print(scipy.stats.shapiro(task_r['Avg Visual Angle']))
print(scipy.stats.shapiro(task_n['Avg Visual Angle']))
print(scipy.stats.mannwhitneyu(task_r['Avg Visual Angle'], task_n['Avg Visual Angle']))

responsive tasks Std Visual Angle: 0.094 ± 0.042
normal tasks Std Visual Angle: 0.090 ± 0.036
Ttest_indResult(statistic=0.3286659456010799, pvalue=0.7440810666992429)
ShapiroResult(statistic=0.96099454164505, pvalue=0.5096436738967896)
ShapiroResult(statistic=0.96555095911026, pvalue=0.6339988708496094)
MannwhitneyuResult(statistic=229.0, pvalue=0.4854638802919542)
responsive tasks Avg Visual Angle: 0.657 ± 0.036
normal tasks Avg Visual Angle: 0.672 ± 0.033
Ttest_indResult(statistic=-1.3421314381118035, pvalue=0.18693873790288193)
ShapiroResult(statistic=0.983332633972168, pvalue=0.9597876071929932)
ShapiroResult(statistic=0.9834082722663879, pvalue=0.9655782580375671)
MannwhitneyuResult(statistic=181.0, pvalue=0.11455094203252936)


In [46]:
result = pandas.merge(weatherTasks, numpadTasks, on=["userID"])
result = pandas.merge(result, articleTasks, on=["userID"])

result

result["Total_Guesses"] = result["Guesses_x"] + result["Guesses_y"] + result["Guesses"]
result


,userID,Mode_x,Application_x,Time_x,Guesses_x,Avg Dist_x,Avg Ratio_x,Time_To_First_Guess,Std Dist_x,Avg Visual Angle_x,...,Mode,Application,Time,Guesses,Avg Dist,Avg Ratio,Std Dist,Avg Visual Angle,Std Visual Angle,Total_Guesses
0,61,Responsive,Numpad,53253,22,0.68327,0.733059,NaN,0.108142,0.616824,...,Responsive,Article,10803,4,0.607538,0.571814,0.015383,0.507244,0.019267,28
1,47,Responsive,Numpad,60275,25,0.648338,0.775888,NaN,0.243236,0.662433,...,Responsive,Article,35084,4,0.620948,0.568456,0.149066,0.521743,0.091895,31
2,63,Responsive,Numpad,79741,26,0.603662,0.831513,NaN,0.148777,0.676065,...,Responsive,Article,55029,7,0.633996,0.544008,0.059207,0.494501,0.038507,35
3,67,Responsive,Numpad,87546,26,0.655263,0.849463,NaN,0.271355,0.685967,...,Responsive,Article,27320,4,0.557659,0.617192,0.07563,0.549192,0.054026,32
4,69,Responsive,Numpad,96772,41,0.63124,0.806009,NaN,0.126382,0.659158,...,Responsive,Article,24510,4,0.636638,0.538416,0.063396,0.491693,0.031233,47
5,41,Responsive,Numpad,100982,30,0.696692,0.756848,NaN,0.065683,0.613189,...,Responsive,Article,20087,4,0.590952,0.589497,0.056277,0.518533,0.028146,36
6,59,Responsive,Numpad,101432,31,0.636548,0.790156,NaN,0.04349,0.641487,...,Responsive,Article,31702,5,0.671707,0.513742,0.06667,0.467884,0.02925,38
7,45,Responsive,Numpad,103221,26,0.759222,0.658577,NaN,0.228755,0.585326,...,Responsive,Article,58152,5,0.597223,0.579941,0.040709,0.517105,0.030428,33
8,43,Responsive,Numpad,112282,36,0.598024,0.828073,NaN,0.141927,0.678926,...,Responsive,Article,108861,12,0.617006,0.581528,0.036322,0.50962,0.02442,52
9,51,Responsive,Numpad,115233,34,0.63134,0.789721,NaN,0.147371,0.659834,...,Responsive,Article,24929,5,0.665112,0.518148,0.06518,0.474154,0.041434,42


In [569]:

task_r = result.loc[result["Mode"] == "Responsive"]
task_n = result.loc[result["Mode"] == "Static"]

taskRAvgTime = task_r['Total_Guesses'].mean()
taskRStd = task_r['Total_Guesses'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Total_Guesses'].mean()
taskNStd = task_n['Total_Guesses'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Total_Guesses'], task_n['Total_Guesses']))

responsive tasks Time: 52.136 ± 27.968
normal tasks Time: 47.190 ± 17.904
Ttest_indResult(statistic=0.6869031512323818, pvalue=0.49601084095890047)


In [570]:
responsive = dataFrame.loc[dataFrame["Mode"] == "Responsive"]
responsive

visualAngleDf = pandas.DataFrame(columns=['userID', 'Application' , 'Task', 'Time', 'Guesses', 'Time Spent in VA1', 'Time Spent in VA2', 'Time Spent in VA3', 'Time Spent in VA4', 'Time Spent in VA5', 'Time Spent in VA6', 'Time Spent in VA7', 'Time Spent in VA8', 'Time Spent in VA9', 'Default VA' ])

visualAngleDf["userID"] = responsive["userID"]
visualAngleDf["Task"] = responsive["Task"]
visualAngleDf["Application"] = responsive["Application"]
visualAngleDf["Time"] = responsive["Time"]
visualAngleDf["Guesses"] = responsive["Guesses"]

mask = visualAngleDf["Time"] > 0

visualAngleDf.loc[mask, 'Time Spent in VA1'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA2'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA3'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA4'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA5'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA6'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA7'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA8'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA9'] = 0
visualAngleDf.loc[mask, 'Default VA'] = 0

In [571]:
for i in visualAngleDf.index:
    userID = visualAngleDf["userID"][i]
    task = visualAngleDf["Task"][i]
    application = visualAngleDf["Application"][i]
    task_num = task - 1

    if(application == "Numpad"):
        task_num += 4

    if(application == "Weather"):
        task_num += 6
    #need math to convert task and app to task_number

    responsiveSession = sessionsDf.loc[sessionsDf["userID"] == userID]

    startEvent = startEvents.loc[(startEvents["userID"] == userID) & (startEvents["task_number"] == task_num)]
    startTime = startEvent["unixTime"].to_numpy()[0]
    endEvent = endEvents.loc[(endEvents["userID"] == userID) & (endEvents["task_number"] == task_num)]
    endTime = endEvent["unixTime"].to_numpy()[0]
    framesUnpacked = list(itertools.chain(*responsiveSession["frames"]))
    sessionFramesDf = pandas.DataFrame(framesUnpacked)

    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] >= startTime]
    #sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= endTime + 100]
    sessionFramesDf = sessionFramesDf.sort_values("unixTime")

    lastFrame = sessionFramesDf.loc[sessionFramesDf["experimentEvents"].str.len() != 0]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] >= endTime]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] <= endTime + 600]

    if(lastFrame.shape[0] != 1):
        print(endTime)
        print(lastFrame.shape[0])
        print(lastFrame["unixTime"].to_numpy())

    lastFrameTime = lastFrame["unixTime"].to_numpy()[0]
    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= lastFrameTime]

    responsiveData = pandas.DataFrame(list(itertools.chain(sessionFramesDf["responsiveData"])))

    ratio = pandas.DataFrame(columns=['ratio', 'unixTime'])
    ratio['ratio'] = responsiveData['ratio']
    ratio["unixTime"] = sessionFramesDf["unixTime"].to_numpy()

    mask = (visualAngleDf["userID"] == userID) & (visualAngleDf["Task"] == task) & (visualAngleDf["Application"] == application)
    total_time = 0

    if application == "Article":
        VA1 = .35523
        VA2 = .333333
        VA3 = .4736839
        VA4 = .64285685

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA2"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA2"
        if task == 3:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 4:
            visualAngleDf.loc[mask, "Default VA"] = "VA4"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time

        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time


    if application == "Numpad":
        VA1 = .27497517
        VA2 = .384615
        VA3 = .40723984
        VA4 = 1.0442047
        VA5 = 1.20192313

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA5"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time
            if(ratio['ratio'][i] >= VA5):
                visualAngleDf.loc[mask, "Time Spent in VA5"] += time
        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time
        if(ratio['ratio'][i] >= VA5):
            visualAngleDf.loc[mask, "Time Spent in VA5"] += time

    if application == "Weather":
        VA1 = .312891085
        VA2 = .312500
        VA3 = .425170065
        VA4 = .62500
        VA5 = .744047
        VA6 = .961538
        VA7 = 1.1363636
        VA8 = 1.19047618
        VA9 = 1.470588287

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA5"
        if task == 3:
            visualAngleDf.loc[mask, "Default VA"] = "VA8"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time
            if(ratio['ratio'][i] < VA6 and ratio['ratio'][i] >= VA5):
                visualAngleDf.loc[mask, "Time Spent in VA5"] += time
            if(ratio['ratio'][i] < VA7 and ratio['ratio'][i] >= VA6):
                visualAngleDf.loc[mask, "Time Spent in VA6"] += time
            if(ratio['ratio'][i] < VA8 and ratio['ratio'][i] >= VA7):
                visualAngleDf.loc[mask, "Time Spent in VA7"] += time
            if(ratio['ratio'][i] < VA9 and ratio['ratio'][i] >= VA8):
                visualAngleDf.loc[mask, "Time Spent in VA8"] += time
            if(ratio['ratio'][i] >= VA9):
                visualAngleDf.loc[mask, "Time Spent in VA9"] += time

        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time
        if(ratio['ratio'][i] < VA6 and ratio['ratio'][i] >= VA5):
            visualAngleDf.loc[mask, "Time Spent in VA5"] += time
        if(ratio['ratio'][i] < VA7 and ratio['ratio'][i] >= VA6):
            visualAngleDf.loc[mask, "Time Spent in VA6"] += time
        if(ratio['ratio'][i] < VA8 and ratio['ratio'][i] >= VA7):
            visualAngleDf.loc[mask, "Time Spent in VA7"] += time
        if(ratio['ratio'][i] < VA9 and ratio['ratio'][i] >= VA8):
            visualAngleDf.loc[mask, "Time Spent in VA8"] += time
        if(ratio['ratio'][i] >= VA9):
            visualAngleDf.loc[mask, "Time Spent in VA9"] += time

    if(total_time != visualAngleDf.loc[mask, "Time"].to_numpy()[0]):
        print("AHHHHHH")
        print(f"userid {userID}")
        print(f"app {application}")
        print(f"task {task}")
        print(f"total_time: {total_time}")
        print(visualAngleDf.loc[mask, "Time"].to_numpy()[0])



In [572]:

visualAngleDf = visualAngleDf.sort_values(by=['Application', 'Task', 'Time'])
visualAngleDf.to_csv("responsiveARTimeSpentInVA.csv")